# Datahour Webinar with Seb Duerr

This Jupyter notebook shows how to query a sqlite database with human questions

## 1) Ingestion

In [1]:
from data.ingestion.prepare_spider_data import prepare_spider_data_faiss
from data.ingestion.prepare_netflix_data import prepare_netflix_data
from data.ingestion.prepare_nltk import download_nltk_packages

SQL_NETFLIX_DATA = "data/datasets/netflix.csv"
VECTOR_SPIDER_DATA = "data/datasets/spider.tsv"
SQL_DATABASE_LOCATION = "data/databases/netflix.db"
VECTOR_DATABASE_LOCATION = "data/databases/spider" 

In [2]:
prepare_netflix_data(data_location = SQL_NETFLIX_DATA, database_location = SQL_DATABASE_LOCATION)
prepare_spider_data_faiss(data_location = VECTOR_SPIDER_DATA, database_location = VECTOR_DATABASE_LOCATION)
download_nltk_packages()

## Ingesting SQLite database with Netflix data. ##
2024-02-04 08:55:42,434 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-04 08:55:42,435 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Netflix")
2024-02-04 08:55:42,435 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-02-04 08:55:42,435 INFO sqlalchemy.engine.Engine COMMIT
2024-02-04 08:55:42,468 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-02-04 08:55:42,489 INFO sqlalchemy.engine.Engine INSERT INTO "Netflix" (uuid, week, category, weekly_rank, show_title, season_title, weekly_hours_viewed, runtime, weekly_views, cumulative_weeks_in_top_10, is_staggered_launch) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2024-02-04 08:55:42,489 INFO sqlalchemy.engine.Engine [generated in 0.01077s] (('af2edad3-b73e-4ad6-9620-ce75d92db5d9', '2023-12-17', 'Films (English)', 1, 'Leave the World Behind', nan, 106200000, 2.3667, 44900000.0, 2, 0), ('4c5361bd-d698-41eb-b0bd-b2d3c91f2b95', '2023-12-17', 'Films (English)', 2, 'Chicken Run: Daw

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/sebastianduerr/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sebastianduerr/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sebastianduerr/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sebastianduerr/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 2) Inference

In [3]:
from src.compose_prompt import compose_prompt
from src.async_generate_sql_queries import generate_sql_query_from_prompt
from src.async_call_sql_endpoint import execute_sql_query

def ai_translate_question_to_sql(question) -> str:
    # Step 1: Compose a prompt using similar queries
    prompt = compose_prompt(question)
    # Step 2: Generate multiple SQL queries
    sql_queries = generate_sql_query_from_prompt(prompt)
    # Step 3: Identifying and returning the most consistent result
    sql_query, query_result = execute_sql_query(sql_queries)
    
    return {"sql_query": sql_query, "query_result": query_result}

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
question = 'What was the most popular show?'
ai_translate_question_to_sql(question)

{'sql_query': ['show_title'], 'query_result': [('Extraction 2',)]}

### Vector Similarity

![image](src/static/similarity.png)

In [5]:
from nltk.tokenize import word_tokenize
from rich import print
from nltk import pos_tag
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()
vector_database = FAISS.load_local(f"{VECTOR_DATABASE_LOCATION}/faiss_index", embeddings)

In [6]:
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

def mask_human_question(question):
    tokens = word_tokenize(question)
    pos_tags = pos_tag(tokens)
    
    masked_question = []
    for word, tag in pos_tags:
        if tag.startswith('NN'): masked_question.append('<mask>')
        elif is_number(word): masked_question.append('<unk>')
        else: masked_question.append(word)
    return ' '.join(masked_question)

def compose_prompt(question: str) -> str:
    schema =  "CREATE TABLE \"Netflix\" (\n\tuuid VARCHAR NOT NULL, \n\tweek DATE, \n\tcategory VARCHAR, \n\tweekly_rank INTEGER, \n\tshow_title VARCHAR, \n\tseason_title VARCHAR, \n\tweekly_hours_viewed INTEGER, \n\truntime FLOAT, \n\tweekly_views INTEGER, \n\tcumulative_weeks_in_top_10 INTEGER, \n\tis_staggered_launch BOOLEAN, \n\tepisode_launch_detail TEXT, \n\tPRIMARY KEY (uuid)\n)"
    masked_human_question = mask_human_question(question)
    similar_query_context = vector_database.similarity_search(query=masked_human_question, k=5)
    related_queries = "\n".join([Document.metadata['Pairs'] for Document in similar_query_context])
    prompt = f"""
#### Generate a sqlite conform query only and with no explanation\n
/* Schema & Types: {schema} */

/* Some example questions and corresponding SQL queries are provided based on similar problems:\n
{related_queries}
/* Answer the following: {question} */\n
    """
    return prompt

In [8]:
question = 'What was the most popular show?'
prompt = compose_prompt(question)
print(prompt)

#### Generate a sqlite conform query only and with no explanation

/* Schema & Types: CREATE TABLE "Netflix" (
        uuid VARCHAR NOT NULL, 
        week DATE, 
        category VARCHAR, 
        weekly_rank INTEGER, 
        show_title VARCHAR, 
        season_title VARCHAR, 
        weekly_hours_viewed INTEGER, 
        runtime FLOAT, 
        weekly_views INTEGER, 
        cumulative_weeks_in_top_10 INTEGER, 
        is_staggered_launch BOOLEAN, 
        episode_launch_detail TEXT, 
        PRIMARY KEY (uuid)
) */

/* Some example questions and corresponding SQL queries are provided based on similar problems:

/* Answer the following: What was the most popular position at tryouts? */
SELECT pPos FROM tryout GROUP BY pPos ORDER BY count(*) DESC LIMIT 1

/* Answer the following: What is the most used instrument? */
SELECT instrument FROM instruments GROUP BY instrument ORDER BY count(*) DESC LIMIT 1

/* Answer the following: What is the most popular first name of the actors? */
SELECT first_name FROM actor GROUP BY first_name ORDER BY count(*) DESC LIMIT 1

/* Answer the following: What are the teams with the most technicians? */
SELECT Team FROM technician GROUP BY Team ORDER BY COUNT(*) DESC LIMIT 1

/* Answer the following: What is the most popular full name of the actors? */
SELECT first_name ,  last_name FROM actor GROUP BY first_name ,  last_name ORDER BY count(*) DESC LIMIT 1

/* Answer the following: What was the most popular show? */

### Vector Similarity Examples

#### Umasked

In [9]:
question_based_similar_documents = vector_database.max_marginal_relevance_search(query=question, k=5)
print(question_based_similar_documents)

[
    Document(
        page_content='What is the name of the most popular plane?',
        metadata={
            'Questions': 'What is the name of the most popular plane?',
            'Queries': 'SELECT plane_name FROM pilotskills GROUP BY plane_name ORDER BY count(*) DESC LIMIT 1',
            'Pairs': '/* Answer the following: What is the name of the most popular plane? */\nSELECT plane_name 
FROM pilotskills GROUP BY plane_name ORDER BY count(*) DESC LIMIT 1\n'
        }
    ),
    Document(
        page_content='What are the names and original air dates of tv shows?',
        metadata={
            'Questions': 'What are the names and original air dates of tv shows?',
            'Queries': 'SELECT tv_show_name ,  Original_Airdate FROM tv_show',
            'Pairs': '/* Answer the following: What are the names and original air dates of tv shows? */\nSELECT 
tv_show_name ,  Original_Airdate FROM tv_show\n'
        }
    ),
    Document(
        page_content='Show the people that have been <mask> the most times.',
        metadata={
            'Questions': 'Show the people that have been governor the most times.',
            'Queries': 'SELECT Governor FROM party GROUP BY Governor ORDER BY COUNT(*) DESC LIMIT 1',
            'Pairs': '/* Answer the following: Show the people that have been governor the most times. */\nSELECT 
Governor FROM party GROUP BY Governor ORDER BY COUNT(*) DESC LIMIT 1\n'
        }
    ),
    Document(
        page_content='Which store owns most items?',
        metadata={
            'Questions': 'Which store owns most items?',
            'Queries': 'SELECT store_id FROM inventory GROUP BY store_id ORDER BY count(*) DESC LIMIT 1',
            'Pairs': '/* Answer the following: Which store owns most items? */\nSELECT store_id FROM inventory 
GROUP BY store_id ORDER BY count(*) DESC LIMIT 1\n'
        }
    ),
    Document(
        page_content='What is the id of the event with the most participants?',
        metadata={
            'Questions': 'What is the id of the event with the most participants?',
            'Queries': 'SELECT Event_ID FROM Participants_in_Events GROUP BY Event_ID ORDER BY count(*) DESC LIMIT 
1',
            'Pairs': '/* Answer the following: What is the id of the event with the most participants? */\nSELECT 
Event_ID FROM Participants_in_Events GROUP BY Event_ID ORDER BY count(*) DESC LIMIT 1\n'
        }
    )
]

In [10]:
unmasked_question_related_queries = "\n".join([Document.metadata['Pairs'] for Document in question_based_similar_documents])
print(unmasked_question_related_queries)

/* Answer the following: What is the name of the most popular plane? */
SELECT plane_name FROM pilotskills GROUP BY plane_name ORDER BY count(*) DESC LIMIT 1

/* Answer the following: What are the names and original air dates of tv shows? */
SELECT tv_show_name ,  Original_Airdate FROM tv_show

/* Answer the following: Show the people that have been governor the most times. */
SELECT Governor FROM party GROUP BY Governor ORDER BY COUNT(*) DESC LIMIT 1

/* Answer the following: Which store owns most items? */
SELECT store_id FROM inventory GROUP BY store_id ORDER BY count(*) DESC LIMIT 1

/* Answer the following: What is the id of the event with the most participants? */
SELECT Event_ID FROM Participants_in_Events GROUP BY Event_ID ORDER BY count(*) DESC LIMIT 1

#### Masked

In [11]:
masked_human_question = mask_human_question(question)
print(masked_human_question)

What was the most popular <mask> ?

In [12]:
similar_query_context = vector_database.similarity_search(query=masked_human_question, k=5)
print(similar_query_context)

[
    Document(
        page_content='What was the most popular position at <mask>?',
        metadata={
            'Questions': 'What was the most popular position at tryouts?',
            'Queries': 'SELECT pPos FROM tryout GROUP BY pPos ORDER BY count(*) DESC LIMIT 1',
            'Pairs': '/* Answer the following: What was the most popular position at tryouts? */\nSELECT pPos FROM 
tryout GROUP BY pPos ORDER BY count(*) DESC LIMIT 1\n'
        }
    ),
    Document(
        page_content='What is the most used <mask>?',
        metadata={
            'Questions': 'What is the most used instrument?',
            'Queries': 'SELECT instrument FROM instruments GROUP BY instrument ORDER BY count(*) DESC LIMIT 1',
            'Pairs': '/* Answer the following: What is the most used instrument? */\nSELECT instrument FROM 
instruments GROUP BY instrument ORDER BY count(*) DESC LIMIT 1\n'
        }
    ),
    Document(
        page_content='What is the most popular first name of the <mask>?',
        metadata={
            'Questions': 'What is the most popular first name of the actors?',
            'Queries': 'SELECT first_name FROM actor GROUP BY first_name ORDER BY count(*) DESC LIMIT 1',
            'Pairs': '/* Answer the following: What is the most popular first name of the actors? */\nSELECT 
first_name FROM actor GROUP BY first_name ORDER BY count(*) DESC LIMIT 1\n'
        }
    ),
    Document(
        page_content='What are the <mask> with the most <mask>?',
        metadata={
            'Questions': 'What are the teams with the most technicians?',
            'Queries': 'SELECT Team FROM technician GROUP BY Team ORDER BY COUNT(*) DESC LIMIT 1',
            'Pairs': '/* Answer the following: What are the teams with the most technicians? */\nSELECT Team FROM 
technician GROUP BY Team ORDER BY COUNT(*) DESC LIMIT 1\n'
        }
    ),
    Document(
        page_content='What is the most popular full name of the <mask>?',
        metadata={
            'Questions': 'What is the most popular full name of the actors?',
            'Queries': 'SELECT first_name ,  last_name FROM actor GROUP BY first_name ,  last_name ORDER BY 
count(*) DESC LIMIT 1',
            'Pairs': '/* Answer the following: What is the most popular full name of the actors? */\nSELECT 
first_name ,  last_name FROM actor GROUP BY first_name ,  last_name ORDER BY count(*) DESC LIMIT 1\n'
        }
    )
]

In [13]:
related_queries = "\n".join([Document.metadata['Pairs'] for Document in similar_query_context])
print(related_queries)

/* Answer the following: What was the most popular position at tryouts? */
SELECT pPos FROM tryout GROUP BY pPos ORDER BY count(*) DESC LIMIT 1

/* Answer the following: What is the most used instrument? */
SELECT instrument FROM instruments GROUP BY instrument ORDER BY count(*) DESC LIMIT 1

/* Answer the following: What is the most popular first name of the actors? */
SELECT first_name FROM actor GROUP BY first_name ORDER BY count(*) DESC LIMIT 1

/* Answer the following: What are the teams with the most technicians? */
SELECT Team FROM technician GROUP BY Team ORDER BY COUNT(*) DESC LIMIT 1

/* Answer the following: What is the most popular full name of the actors? */
SELECT first_name ,  last_name FROM actor GROUP BY first_name ,  last_name ORDER BY count(*) DESC LIMIT 1

### Generate the SQL query with an LLM

In [14]:
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()
import os

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

def openai_generate_sql_query(prompt: str) -> str:    
    chat_completion = client.chat.completions.create(messages=[{"role": "user", "content": prompt}], model="gpt-4-1106-preview", temperature=0)
    sql_query = chat_completion.choices[0].message.content
    cleaned_sql_query = sql_query.replace('`', '').replace('sql\n', '')
    return cleaned_sql_query

sql_query = openai_generate_sql_query(prompt)
print(sql_query)

SELECT show_title FROM Netflix GROUP BY show_title ORDER BY SUM(weekly_hours_viewed) DESC LIMIT 1

### Querying the SQLite Database

In [15]:
import sqlite3

DATABASE_URL = "data/databases/netflix.db"

def call_sql_endpoint(sql_query: str):
    try:
        with sqlite3.connect(DATABASE_URL) as connection:
            cursor = connection.cursor()
            cursor.execute(sql_query)
            columns = [desc[0] for desc in cursor.description]
            rows = cursor.fetchall()
            cursor.close()
            return columns, rows
    except Exception as e:
        print(f"Unexpected error: {e}")
        raise RuntimeError("An unexpected error occurred. Please contact support if the problem persists.") from e

In [16]:
result = call_sql_endpoint(sql_query)
print(question)
print(result)

What was the most popular show?

(['show_title'], [('Stranger Things',)])

## Next Steps

As Next Steps we could:

- We can generate a service from this and implement `self-consistency`.
- `Validate` our results and seek areas for improvement.
- We could explain or visualize the response.